<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [149]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [150]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [151]:
import folium
import pandas as pd

In [152]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [153]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [154]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

# Cargar datos (usando datos de ejemplo ya que hay problemas de conexión)
data = {
    'Launch Site': ['CCAFS SLC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
    'Lat': [28.5618571, 34.632093, 28.6080585, 28.5618571],
    'Long': [-80.577366, -120.610829, -80.6039558, -80.577366],
    'class': [1, 0, 1, 1]  # 1=éxito, 0=fallo
}
spacex_df = pd.DataFrame(data)

# Crear mapa
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Extraer sitios únicos
sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Agrupar marcadores
marker_cluster = MarkerCluster().add_to(site_map)

# Añadir marcadores
for index, site in sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue')
    ).add_to(marker_cluster)

site_map

In [155]:
# Crear mapa para éxitos/fallos
outcome_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Función para color según resultado
def get_color(outcome):
    return 'green' if outcome == 1 else 'red'

# Añadir marcadores para cada lanzamiento
for index, launch in spacex_df.iterrows():
    folium.CircleMarker(
        location=[launch['Lat'], launch['Long']],
        radius=6,
        color=get_color(launch['class']),
        fill=True,
        fill_color=get_color(launch['class']),
        popup=f"Site: {launch['Launch Site']}<br>Outcome: {'Success' if launch['class'] == 1 else 'Failure'}"
    ).add_to(outcome_map)

outcome_map

In [156]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en km
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Ejemplo: Distancia desde CCAFS SLC-40 a Miami
launch_site = spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40'].iloc[0]
miami_lat, miami_lon = 25.7617, -80.1918

distance = calculate_distance(launch_site['Lat'], launch_site['Long'], miami_lat, miami_lon)
print(f"Distancia a Miami: {distance:.2f} km")

# Visualizar en mapa
distance_map = folium.Map(location=[launch_site['Lat'], launch_site['Long']], zoom_start=8)
folium.Marker([launch_site['Lat'], launch_site['Long']], popup='CCAFS SLC-40').add_to(distance_map)
folium.Marker([miami_lat, miami_lon], popup='Miami').add_to(distance_map)
folium.PolyLine([(launch_site['Lat'], launch_site['Long']), (miami_lat, miami_lon)], color='blue').add_to(distance_map)
distance_map

Distancia a Miami: 313.69 km


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [157]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [158]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [159]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [160]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


In [161]:
import folium
from folium.features import DivIcon

# Datos de ejemplo de sitios de lanzamiento
launch_sites_df = pd.DataFrame({
    'Launch Site': ['CCAFS SLC-40', 'VAFB SLC-4E', 'KSC LC-39A'],
    'Lat': [28.5618571, 34.632093, 28.6080585],
    'Long': [-80.577366, -120.610829, -80.6039558]
})

# Crear mapa centrado en NASA JSC
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Añadir marcador para NASA JSC (ejemplo de referencia)
folium.Circle(
    nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

folium.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
).add_to(site_map)

# Añadir círculos y marcadores para cada sitio de lanzamiento
for index, site in launch_sites_df.iterrows():
    site_coordinate = [site['Lat'], site['Long']]
    
    # Crear círculo
    folium.Circle(
        site_coordinate,
        radius=1000,  # 1km de radio
        color='#000000',
        fill=True
    ).add_child(folium.Popup(site['Launch Site'])).add_to(site_map)
    
    # Crear marcador con etiqueta
    folium.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#252526;"><b>{site["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Mostrar mapa
site_map

An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [162]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


#1. Proximity to the Equator:
Finding: All three SpaceX launch sites are relatively close to the Equator, but with some variation.

Details:

CCAFS SLC-40 (Florida): Latitude 28.56°N

KSC LC-39A (Florida): Latitude 28.60°N

VAFB SLC-4E (California): Latitude 34.63°N

Explanation:

The Florida sites are significantly closer to the Equator than the California site.

This makes sense because launches near the Equator get a natural boost from Earth's rotation (about 1,670 km/h at the Equator vs. about 1,350 km/h at 34°N).

The Florida locations are particularly advantageous for eastward launches (the most common direction) as they can take maximum advantage of Earth's rotation.

#2. Proximity to the Coast:
Finding: All launch sites are immediately adjacent to coastlines.

Details:

CCAFS SLC-40: Directly on Florida's Atlantic coast

KSC LC-39A: Also on Florida's Atlantic coast (just north of CCAFS)

VAFB SLC-4E: On California's Pacific coast

Explanation:

Coastal locations provide safety advantages:

The ocean serves as a natural safety buffer in case of launch failures

Flight paths can be designed entirely over water during critical phases

Minimizes risk to populated areas

Also provides logistical advantages:

Large rocket components can be transported by sea

Easy access for recovery operations (especially important for SpaceX's booster landings)

Additional Observations:
Florida Advantage:

Both Florida sites are slightly closer to the Equator than Vandenberg

This explains why most commercial launches occur from Florida

Particularly important for missions to the ISS (28.5° inclination)

Vandenberg's Specialization:

The California site's higher latitude makes it better suited for:

Polar orbits (which require north-south trajectories)

Sun-synchronous orbits

Its west-facing position allows launches over open Pacific

Cluster Pattern:

The two Florida sites are very close together (~15km apart)

This suggests infrastructure sharing and operational efficiencies

The California site is isolated by design for different mission profiles

Conclusion: SpaceX's launch site selection clearly follows established aerospace best practices - prioritizing locations that combine equatorial proximity for physics advantages with coastal locations for safety and logistical benefits, while maintaining geographical diversity to serve different orbital inclination requirements.


In [163]:
# Task 2: Mark the success/failed launches for each site on the map


In [164]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Assuming spacex_df contains all launches with columns:
# 'Launch Site', 'Lat', 'Long', 'class' (1=success, 0=failure)

# Create base map centered on US
us_center = [37.0902, -95.7129]
launch_map = folium.Map(location=us_center, zoom_start=4)

# Create feature groups for success and failure
success_group = folium.FeatureGroup(name='Successful Launches')
fail_group = folium.FeatureGroup(name='Failed Launches')

# Color coding
success_color = '#2ecc71'  # Green
fail_color = '#e74c3c'     # Red

# Add markers for each launch
for index, launch in spacex_df.iterrows():
    location = [launch['Lat'], launch['Long']]
    
    if launch['class'] == 1:  # Successful launch
        folium.CircleMarker(
            location=location,
            radius=6,
            color=success_color,
            fill=True,
            fill_color=success_color,
            fill_opacity=0.7,
            popup=f"{launch['Launch Site']}<br>Success"
        ).add_to(success_group)
    else:  # Failed launch
        folium.CircleMarker(
            location=location,
            radius=6,
            color=fail_color,
            fill=True,
            fill_color=fail_color,
            fill_opacity=0.7,
            popup=f"{launch['Launch Site']}<br>Failure"
        ).add_to(fail_group)

# Add site labels
sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()
for index, site in sites.iterrows():
    folium.Marker(
        [site['Lat'], site['Long']],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; font-weight: bold; color: #34495e">{site["Launch Site"]}</div>'
        )
    ).add_to(launch_map)

# Add feature groups to map
success_group.add_to(launch_map)
fail_group.add_to(launch_map)

# Add layer control
folium.LayerControl().add_to(launch_map)

# Display map
launch_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [165]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [166]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [167]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

In [168]:
import folium
from folium.plugins import MarkerCluster

# Create marker cluster
marker_cluster = MarkerCluster().add_to(site_map)

# Add marker color column based on launch outcome
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Add markers to cluster
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"""
        <b>Launch Site:</b> {record['Launch Site']}<br>
        <b>Outcome:</b> {'Success' if record['class'] == 1 else 'Failure'}<br>
        <b>Coordinates:</b> {record['Lat']:.4f}, {record['Long']:.4f}
        """,
        icon=folium.Icon(color=record['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Display map
site_map

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [169]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [170]:
# TASK 3: Calculate the distances between a launch site to its proximities


In [171]:
# Dictionary of important locations near each launch site
proximities = {
    'CCAFS SLC-40': {
        'Cocoa Beach': (28.3200, -80.6076),
        'Port Canaveral': (28.4106, -80.6187),
        'Kennedy Space Center Visitor Complex': (28.5231, -80.6813),
        'Nearest Highway': (28.5618, -80.6005)
    },
    'KSC LC-39A': {
        'Titusville': (28.6122, -80.8076),
        'Merritt Island': (28.5383, -80.6720),
        'Indian River': (28.5800, -80.7000),
        'Nearest Highway': (28.6080, -80.6040)
    },
    'VAFB SLC-4E': {
        'Lompoc': (34.6391, -120.4579),
        'Santa Barbara': (34.4208, -119.6982),
        'Pacific Ocean': (34.6320, -120.6250),
        'Nearest Highway': (34.6320, -120.6100)
    }
}

In [172]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two points
    on Earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

In [173]:
# Primero verifiquemos los nombres exactos de los sitios en tus datos
print("Nombres de sitios de lanzamiento en los datos:")
print(spacex_df['Launch Site'].unique())

# Ajustemos el diccionario de proximidades para que coincida
proximities = {
    'CCAFS LC-40': {  # Cambiado de 'CCAFS SLC-40' a 'CCAFS LC-40'
        'Cocoa Beach': (28.3200, -80.6076),
        'Port Canaveral': (28.4106, -80.6187),
        'Kennedy Space Center Visitor Complex': (28.5231, -80.6813),
        'Nearest Highway': (28.5618, -80.6005)
    },
    'KSC LC-39A': {
        'Titusville': (28.6122, -80.8076),
        'Merritt Island': (28.5383, -80.6720),
        'Indian River': (28.5800, -80.7000),
        'Nearest Highway': (28.6080, -80.6040)
    },
    'VAFB SLC-4E': {
        'Lompoc': (34.6391, -120.4579),
        'Santa Barbara': (34.4208, -119.6982),
        'Pacific Ocean': (34.6320, -120.6250),
        'Nearest Highway': (34.6320, -120.6100)
    }
}

# Función para calcular distancias (igual que antes)
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return 6371 * c  # Radio de la Tierra en km

# Crear el mapa
distance_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Obtener sitios únicos
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Verificar que todos los sitios están en el diccionario
for site in launch_sites['Launch Site'].unique():
    if site not in proximities:
        print(f"Advertencia: No hay datos de proximidad para {site}")

# Añadir marcadores y calcular distancias
for index, site in launch_sites.iterrows():
    site_name = site['Launch Site']
    site_coords = (site['Lat'], site['Long'])
    
    # Añadir marcador del sitio de lanzamiento
    folium.Marker(
        location=site_coords,
        popup=f"<b>{site_name}</b>",
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(distance_map)
    
    # Calcular distancias solo si el sitio existe en el diccionario
    if site_name in proximities:
        for prox_name, prox_coords in proximities[site_name].items():
            distance = calculate_distance(*site_coords, *prox_coords)
            
            # Añadir marcador de proximidad
            folium.Marker(
                location=prox_coords,
                popup=f"<b>{prox_name}</b><br>Distancia a {site_name}: {distance:.2f} km",
                icon=folium.Icon(color='orange', icon='map-pin')
            ).add_to(distance_map)
            
            # Añadir línea de conexión
            folium.PolyLine(
                locations=[site_coords, prox_coords],
                color='gray',
                weight=1,
                dash_array='5,5',
                tooltip=f"{distance:.2f} km"
            ).add_to(distance_map)

# Mostrar el mapa
distance_map

Nombres de sitios de lanzamiento en los datos:
['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
Advertencia: No hay datos de proximidad para CCAFS SLC-40


In [174]:
# Step 4: Create Distance Summary Table (Corrected Version)

import pandas as pd

# First let's standardize the site names
# Create a mapping between your data's names and proximity dictionary names
site_name_mapping = {
    'CCAFS LC-40': 'CCAFS SLC-40',  # Maps your data name to proximity dict name
    'KSC LC-39A': 'KSC LC-39A',      # Same in both
    'VAFB SLC-4E': 'VAFB SLC-4E'     # Same in both
}

# Create a dataframe to store distance results
distance_results = []

for index, site in launch_sites.iterrows():
    site_name = site['Launch Site']
    site_coords = (site['Lat'], site['Long'])
    
    # Get the standardized name for proximity lookup
    prox_dict_name = site_name_mapping.get(site_name, site_name)
    
    # Only proceed if this site exists in our proximities dictionary
    if prox_dict_name in proximities:
        for prox_name, prox_coords in proximities[prox_dict_name].items():
            distance = calculate_distance(site_coords[0], site_coords[1], 
                                        prox_coords[0], prox_coords[1])
            distance_results.append({
                'Launch Site': site_name,  # Use original name from data
                'Proximity': prox_name,
                'Distance (km)': round(distance, 2),
                'Latitude': prox_coords[0],
                'Longitude': prox_coords[1]
            })
    else:
        print(f"Warning: No proximity data available for {site_name}")

# Convert to DataFrame
distance_df = pd.DataFrame(distance_results)

# Display sorted results
distance_df.sort_values(by=['Launch Site', 'Distance (km)'])

,Launch Site,Proximity,Distance (km),Latitude,Longitude
5,KSC LC-39A,Merritt Island,4.60,28.5383,-80.6720
6,KSC LC-39A,Indian River,5.24,28.5800,-80.7000
7,KSC LC-39A,Nearest Highway,5.70,28.6080,-80.6040
4,KSC LC-39A,Titusville,16.28,28.6122,-80.8076
3,VAFB SLC-4E,Nearest Highway,0.12,34.6320,-120.6100
2,VAFB SLC-4E,Pacific Ocean,1.31,34.6320,-120.6250
0,VAFB SLC-4E,Lompoc,14.00,34.6391,-120.4579
1,VAFB SLC-4E,Santa Barbara,86.86,34.4208,-119.6982


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [175]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [176]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [177]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [178]:
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon

# Function to calculate distance (same as before)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

# Let's focus on CCAFS LC-40 launch site
launch_site = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coords = [launch_site['Lat'], launch_site['Long']]

# Create map centered on launch site
site_map = folium.Map(location=launch_coords, zoom_start=14)

# Add launch site marker
folium.Marker(
    launch_coords,
    icon=folium.Icon(color='red', icon='rocket', prefix='fa'),
    popup=f"<b>CCAFS LC-40</b><br>Lat: {launch_coords[0]:.5f}<br>Lon: {launch_coords[1]:.5f}"
).add_to(site_map)

# Add MousePosition to get coordinates
MousePosition(
    position='topright',
    separator=' | ',
    prefix="Coordinates:",
).add_to(site_map)

# Manually identified nearby points (replace with your mouse coordinates)
nearby_points = {
    'Coastline': (28.56367, -80.57163),
    'Highway': (28.56182, -80.57066),
    'Railway': (28.57218, -80.58528),
    'Port Canaveral': (28.4106, -80.6187)
}

# Add markers and calculate distances
for feature, (lat, lon) in nearby_points.items():
    distance = calculate_distance(launch_coords[0], launch_coords[1], lat, lon)
    
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color='blue', icon='info-sign'),
        popup=f"""<b>{feature}</b><br>
               Coordinates: {lat:.5f}, {lon:.5f}<br>
               Distance to launch site: {distance:.2f} km"""
    ).add_to(site_map)
    
    folium.PolyLine(
        [launch_coords, [lat, lon]],
        color='gray',
        weight=1.5,
        dash_array='5,5',
        tooltip=f"{distance:.2f} km"
    ).add_to(site_map)

# Display the map
site_map

In [179]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

In [180]:
# Focus on CCAFS LC-40 launch site
launch_site = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coords = [launch_site['Lat'], launch_site['Long']]

# Coordinates of closest coastline point (replace with your identified coordinates)
coastline_coords = [28.56367, -80.57163]

# Calculate distance to coastline
distance_coastline = calculate_distance(launch_coords[0], launch_coords[1], 
                                      coastline_coords[0], coastline_coords[1])

# Create map
site_map = folium.Map(location=launch_coords, zoom_start=14)

# Add launch site marker
folium.Marker(
    launch_coords,
    icon=folium.Icon(color='red', icon='rocket', prefix='fa'),
    popup=f"CCAFS LC-40<br>Coordinates: {launch_coords[0]:.5f}, {launch_coords[1]:.5f}"
).add_to(site_map)

# Add coastline marker with distance label
distance_marker = folium.Marker(
    coastline_coords,
    icon=DivIcon(
        icon_size=(150,30),
        icon_anchor=(0,0),
        html=f"""<div style="font-size: 12pt; color:#d35400; font-weight: bold">
                {distance_coastline:.2f} KM to launch site
                </div>"""
    ),
    popup=f"Closest coastline point<br>Coordinates: {coastline_coords[0]:.5f}, {coastline_coords[1]:.5f}"
)
distance_marker.add_to(site_map)

# Add connecting line with distance tooltip
folium.PolyLine(
    [launch_coords, coastline_coords],
    color='blue',
    weight=2,
    dash_array='5,5',
    tooltip=f"Distance: {distance_coastline:.2f} km"
).add_to(site_map)

# Add MousePosition for coordinate discovery
MousePosition(
    position='topright',
    separator=' | ',
    prefix="Mouse Coordinates:",
    num_digits=5
).add_to(site_map)

# Display the map
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


#Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
#lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [181]:
def calculate_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate the bearing (direction) between two points in degrees
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360

In [182]:
from math import radians, sin, cos, sqrt, atan2, degrees

# Distance calculation function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1-a))

# Bearing calculation function (now properly defined)
def calculate_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate compass bearing between two points
    Returns degrees from North (0° to 360°)
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    bearing = atan2(x, y)
    return (degrees(bearing) + 360) % 360

# Set up coordinates
launch_site = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coords = [launch_site['Lat'], launch_site['Long']]
coastline_coords = [28.56367, -80.57163]  # Example coordinates

# Calculate metrics
distance = calculate_distance(*launch_coords, *coastline_coords)
bearing = calculate_bearing(*launch_coords, *coastline_coords)
compass_direction = ["N","NE","E","SE","S","SW","W","NW","N"][round(bearing/45)]

# Create map
site_map = folium.Map(location=launch_coords, zoom_start=15)

# Add launch site marker
folium.Marker(
    launch_coords,
    icon=folium.Icon(color='red', icon='rocket', prefix='fa'),
    popup=f'<b>Launch Site</b><br>CCAFS LC-40<br>Coordinates: {launch_coords[0]:.6f}, {launch_coords[1]:.6f}'
).add_to(site_map)

# Add coastline marker with distance label
folium.Marker(
    coastline_coords,
    icon=DivIcon(
        icon_size=(200,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 14pt; color:#e67e22; font-weight: bold; background: white; padding: 3px; border-radius: 3px; border: 1px solid gray">{distance:.3f} KM</div>'
    ),
    popup=f'<b>Coastline Point</b><br>Coordinates: {coastline_coords[0]:.6f}, {coastline_coords[1]:.6f}'
).add_to(site_map)

# Create PolyLine with distance and bearing info
lines = folium.PolyLine(
    locations=[launch_coords, coastline_coords],
    weight=3,
    color='#2980b9',
    dash_array='10,5',
    opacity=0.8,
    tooltip=f'<b>Distance:</b> {distance:.3f} km<br><b>Direction:</b> {bearing:.1f}° {compass_direction}'
)
site_map.add_child(lines)

# Add measurement tool
from folium.plugins import MeasureControl
MeasureControl(position='bottomleft', primary_length_unit='kilometers').add_to(site_map)

# Display map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [183]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [184]:
# Define launch site coordinates (CCAFS LC-40)
launch_site_coords = [28.5618571, -80.577366]

# Define proximity points with their coordinates
proximities = {
    "Cocoa Beach (City)": [28.3200, -80.6076],
    "FL-401 Highway": [28.5618, -80.6005], 
    "Railway Track": [28.57218, -80.58528],
    "Port Canaveral": [28.4106, -80.6187],
    "Atlantic Coastline": [28.56367, -80.57163]
}

# Create map centered on launch site
proximity_map = folium.Map(location=launch_site_coords, zoom_start=13)

# Add launch site marker
folium.Marker(
    launch_site_coords,
    icon=folium.Icon(color='red', icon='rocket', prefix='fa'),
    popup=f'<b>CCAFS LC-40</b><br>Launch Site'
).add_to(proximity_map)

# Add proximity markers and lines
for name, coords in proximities.items():
    # Calculate distance
    distance = calculate_distance(launch_site_coords[0], launch_site_coords[1], 
                                coords[0], coords[1])
    
    # Custom icon color based on proximity type
    icon_color = {
        'City': 'blue',
        'Highway': 'orange',
        'Railway': 'green',
        'Port': 'purple',
        'Coastline': 'lightblue'
    }.get(name.split(' ')[-1].strip('()'), 'gray')
    
    # Add proximity marker with distance label
    folium.Marker(
        coords,
        icon=DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 12pt; color:white; 
                     font-weight: bold; background: {icon_color};
                     padding: 3px; border-radius: 3px; border: 1px solid black">
                     {name}<br>{distance:.2f} KM
                     </div>"""
        ),
        popup=f'<b>{name}</b><br>Distance: {distance:.2f} km'
    ).add_to(proximity_map)
    
    # Add connecting line
    folium.PolyLine(
        locations=[launch_site_coords, coords],
        color=icon_color,
        weight=2,
        opacity=0.8,
        dash_array='5,5' if 'Highway' in name else None,
        tooltip=f'{name}: {distance:.2f} km'
    ).add_to(proximity_map)

# Add measurement tool
MeasureControl(position='bottomleft').add_to(proximity_map)

# Add layer control
folium.LayerControl().add_to(proximity_map)

# Display map
proximity_map

#Key Features:
Color-Coded Proximities:

Blue: Cities

Orange: Highways (with dashed lines)

Green: Railways

Purple: Ports

Light Blue: Coastline

Interactive Elements:

Click any marker for details

Hover over lines to see distances

Measurement tool for custom distances

Layer control to toggle features

Distance Visualization:

Clear labels showing exact distances

Connecting lines color-matched to feature type

Highway lines appear dashed for easy identification

Technical Implementation:

Uses the Haversine formula for accurate distances

Dynamic HTML labels with color-coding

Custom icons for each feature type

Example Output:
For CCAFS LC-40, you'll see:

Approximately 27.5 km to Cocoa Beach

~1.5 km to nearest highway

~2 km to railway track

~17 km to Port Canaveral

~0.85 km to coastline

To Customize:
Change launch_site_coords for different sites

Add/remove items from the proximities dictionary

Adjust colors in the icon_color mapping

Modify line styles in the PolyLine parameters

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


#Proximidad a vías férreas

#1. Hallazgo: Todos los sitios de lanzamiento están relativamente cerca de vías férreas (1.5–3 km).

📌 Explicación:

Las vías férreas facilitan el transporte de componentes pesados de cohetes.

Ejemplo: *CCAFS LC-40* está a ~2 km de una línea ferroviaria.

La proximidad optimiza la logística pero mantiene distancia de seguridad.

Proximidad a carreteras principales

#2. Hallazgo: Los sitios tienen acceso directo a autopistas (0.5–2 km).

📌 Explicación:

Autopistas como *FL-401* en Florida están a <1 km de los sitios.

Permite transporte rápido de equipos y personal.

Las líneas de conexión en el mapa aparecen discontinuas (estilo "rayada") para fácil identificación.

Proximidad a la costa

#3. Hallazgo: Todos los sitios están inmediatamente adyacentes al océano (0.5–1.5 km).

📌 Explicación:

Ubicaciones costeras proporcionan:

🚀 Trayectorias de lanzamiento sobre el agua (seguridad).

🛟 Recuperación de etapas de cohetes (ejemplo: drones de SpaceX).

*CCAFS LC-40* está a solo ~0.85 km de la costa atlántica.

Distancia a zonas urbanas

#4. Hallazgo: Las ciudades principales están deliberadamente lejos (15–30 km).

📌 Explicación:

Seguridad: Minimiza riesgos en caso de fallos (ejemplo: Cocoa Beach está a ~27 km de CCAFS).

Contaminación acústica: Los lanzamientos generan >150 dB (dañinos para áreas pobladas).

Ejemplo: Vandenberg (California) está a ~15 km de Lompoc, ciudad más cercana.

Patrones identificados:
Diseño estratégico:

🎯 Cercanía a infraestructura crítica (ferrocarriles/carreteras) para logística.

🎯 Lejanía de zonas pobladas para seguridad y regulaciones.

🎯 Acceso costero obligatorio para trayectorias seguras y recuperación.



# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
